In [11]:
import geopandas as gpd

## Open Store data

In [35]:
dennys_gdf = gpd.read_file("data/dennys_per_state.gpkg")
dennys_gdf = dennys_gdf[["STUSPS", "DENNYS"]]

In [36]:
gpd.read_file("data/dennys_per_state.gpkg")[["NAME", "per_1m"]].sort_values(
    "per_1m", ascending=False
).head(3)

,NAME,per_1m
12,New Mexico,13.671739
28,Nevada,12.444145
35,Arizona,10.571352


In [37]:
gpd.read_file("data/dennys_per_state.gpkg")[["NAME", "DENNYS"]].sort_values(
    "DENNYS", ascending=False
).head(2)

,NAME,DENNYS
16,California,355.0
15,Texas,195.0


In [38]:
ihops_gdf = gpd.read_file("data/IHOPs_per_state.gpkg")
ihops_gdf = ihops_gdf[["geometry", "STUSPS", "IHOPs"]]

In [39]:
gpd.read_file("data/IHOPs_per_state.gpkg")[["NAME", "per_1m"]].sort_values(
    "per_1m", ascending=False
).head(3)

,NAME,per_1m
26,Kansas,10.164323
38,Maryland,8.685775
2,Oklahoma,8.612455


In [40]:
gpd.read_file("data/IHOPs_per_state.gpkg")[["NAME", "IHOPs"]].sort_values(
    "IHOPs", ascending=False
).head(2)

,NAME,IHOPs
16,California,223.0
15,Texas,216.0


#### Merge Data now

In [14]:
dennys_ihop_gdf = ihops_gdf.merge(dennys_gdf, on="STUSPS", how="left")

In [15]:
dennys_ihop_gdf["IHOPs"] = dennys_ihop_gdf["IHOPs"].astype(int)
dennys_ihop_gdf["DENNYS"] = dennys_ihop_gdf["DENNYS"].astype(int)

In [16]:
dennys_ihop_gdf["IHOP_DENNYS"] = dennys_ihop_gdf["IHOPs"] - dennys_ihop_gdf["DENNYS"]

In [22]:
def more_stores(row) -> str:
    dennys = row["DENNYS"]
    ihop = row["IHOPs"]
    if dennys > ihop:
        return "More Denny's"
    if ihop > dennys:
        return "More IHOPs"
    if (ihop == dennys) and (ihop == 0):
        return "No Locations"
    if ihop == dennys:
        return "Same Amount"

In [23]:
dennys_ihop_gdf["label"] = dennys_ihop_gdf.apply(lambda row: more_stores(row), axis=1)

In [25]:
dennys_ihop_gdf = dennys_ihop_gdf.to_crs(9311)
dennys_ihop_gdf.to_file("data/ihop_dennys.gpkg")